In [ ]:
try:
    from google.colab import files
    uploaded = files.upload()

    import zipfile
    with zipfile.ZipFile("organized_cat_dataset.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Dataset unzipped successfully!")
except:
    print("Running outside Google Colab or skipping file upload.")

Saving organized_cat_dataset.zip to organized_cat_dataset.zip
Dataset unzipped successfully!


In [ ]:
#libraries

import os
from PIL import Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from sklearn.metrics import classification_report

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [ ]:
tfm = transforms.Compose([
    transforms.Resize((224, 224)), # resizes images to 224x224 pixels
    transforms.RandomHorizontalFlip(), # randomly flips and rotates images (for training data augmentation)
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(), # converts images to pytorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Class names
class_names = ['alarmed', 'angry', 'calm', 'pleased']

In [ ]:
# Dataset paths (organized by folders)
train_img_dir = "/content/train_images"
val_img_dir = "/content/valid_images"
test_img_dir = "/content/test_images"

In [ ]:
# Load datasets using ImageFolder
train_set = datasets.ImageFolder(root=train_img_dir, transform=tfm)
val_set = datasets.ImageFolder(root=val_img_dir, transform=tfm)
test_set = datasets.ImageFolder(root=test_img_dir, transform=tfm)

In [ ]:
# Data loaders
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)
test_loader = DataLoader(test_set, batch_size=16)

In [ ]:
# Load pretrained ResNet18
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 output classes
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 128MB/s]


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training (It take around 30 - 35 minutes to finish this training)
for epoch in range(10):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}")

Epoch 1: Loss=77.5014
Epoch 2: Loss=55.7783
Epoch 3: Loss=48.1576
Epoch 4: Loss=43.0115
Epoch 5: Loss=32.4123
Epoch 6: Loss=29.2103
Epoch 7: Loss=24.6822
Epoch 8: Loss=21.2748
Epoch 9: Loss=18.1659
Epoch 10: Loss=14.6755


In [ ]:
# --- Simplified Validation (Accuracy and Loss) ---
if val_loader and val_set and len(val_set) > 0:
    print("\nCalculating Validation Metrics...")
    model.eval()
    correct = 0
    total = 0
    val_running_loss = 0.0 # Initialize validation loss
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Calculate validation loss for this batch
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            # Get predictions for accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if total > 0:
        val_accuracy = 100 * correct / total
        val_epoch_loss = val_running_loss / len(val_set) # Calculate average validation loss
        print(f"Validation Loss: {val_epoch_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.2f}%")
    else:
        print("No data in validation loader to calculate metrics.")
else:
    print("Validation data not loaded or is empty. Skipping validation.")


Calculating Validation Metrics...
Validation Loss: 1.9550
Validation Accuracy: 53.57%


In [ ]:
# Test Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


Classification Report:
              precision    recall  f1-score   support

     alarmed       0.60      0.67      0.63         9
       angry       1.00      0.82      0.90        17
        calm       0.40      0.50      0.44         8
     pleased       0.50      0.50      0.50         2

    accuracy                           0.69        36
   macro avg       0.62      0.62      0.62        36
weighted avg       0.74      0.69      0.71        36



In [ ]:
# # Test Evaluation
# model.eval()
# y_true = []
# y_pred = []
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         y_true.extend(labels.cpu().numpy())
#         y_pred.extend(predicted.cpu().numpy())

In [ ]:
# # Report
# print("\nClassification Report:")
# print(classification_report(y_true, y_pred, target_names=class_names))